In [1]:
# 在原版paper的deepfm算法做了一点改进，即fm_component和deep component输出的都不只是一个数了，而是一个响亮
# 大量参照https://github.com/ChenglongChen/tensorflow-DeepFM
# 他对deep fm进行了一点点优化，虽然数据描述不太明白，但是其数据结构设计得确实好，很精简也非常适合这个算法
# deep fm只能处理one-hot类型的，无法直接处理multi-hot特征
import numpy as np
import tensorflow as tf

In [2]:
class EmbeddingLayer(tf.keras.layers.Layer):
    def __init__(self,feat_dim,field_num,emb_dim,*args,**kwargs):
        super(EmbeddingLayer,self).__init__(*args,**kwargs)
        self.feat_dim=feat_dim
        self.field_num=field_num
        self.emb_dim=emb_dim
        self.emb_layer=tf.keras.layers.Embedding(input_dim=feat_dim,output_dim=emb_dim)

    def call(self, inputs, **kwargs):

        """

        :param inputs: (feat_indices_batch, feat_value_batch)，分为两部分，你可以把他看做一个手动构造的稀疏矩阵
            例如，如果feat_indices_batch的数据为[[1,5,9],[2,7,8]]；feat_value_batch的数据为[[1,1,2.3],[1,1,0.98]]
            假设第一个样本的特征向量是x，那么x[1]=1, x[5]=1, x[9]=2.3，其余位置取值均为0。
            这样构造是因为每个样本都有field_num个field，每个field的取值只有一种（one-hot或者连续值）
            也就是说每个样本都有field_num个不为0的特征维度。而deep fm算法的嵌入方法是对每一个field嵌入，不管是不是连续值都要嵌入，然后再乘以特征取值
            例如x[9]=2.3，那就要从emb_table里找到第9个emb_vector，然后乘以2.3
        :param kwargs:
        :return:
        """

        # feat_indices_batch: [batch_size, field_num]
        # feat_value_batch: [batch_size, field_num]
        feat_indices_batch,feat_value_batch=inputs
        # 两者形状要相同，并且二者的第二个轴取值维度都是field_num个
        assert feat_indices_batch.shape==feat_value_batch.shape
        assert feat_indices_batch.shape[1:]==[self.field_num]

        emb_vectors=self.emb_layer(feat_indices_batch) # [batch_size, field_num, emb_dim]
        feat_value_batch = tf.expand_dims(feat_value_batch,axis=-1) # [batch_size, field_num, 1]

        # broadcast性质 feat_value_batch会被看做[batch_size, field_num, emb_dim]
        emb_vectors = tf.multiply(emb_vectors,feat_value_batch) # [batch_size, field_num, emb_dim]
        return emb_vectors


feat_indices_arr=[np.random.choice(range(10),size=[1,3],replace=False) for _ in range(9)]
feat_indices_arr=np.concatenate(feat_indices_arr,axis=0).astype(np.float32)
print("feat_indices_arr")
print(feat_indices_arr) #[10,3]

feat_vals_arr=np.concatenate((np.ones(shape=[9,2]),
                              np.random.random(size=[9,1])),axis=1).astype(np.float32)
print("\nfeat_vals_arr")
print(feat_vals_arr) # [10,3]

input_ds=tf.data.Dataset.from_tensor_slices((feat_indices_arr,feat_vals_arr))
batched_ds=input_ds.batch(5)
iterator=iter(batched_ds)
input_batch=next(iterator)


print("\nemb vectors")
emb_layer=EmbeddingLayer(feat_dim=10,field_num=3,emb_dim=4)
emb_vectors=emb_layer(input_batch)
print(emb_vectors)


feat_indices_arr
[[2. 9. 3.]
 [2. 1. 3.]
 [9. 4. 6.]
 [7. 5. 4.]
 [3. 1. 8.]
 [8. 9. 1.]
 [1. 0. 2.]
 [4. 9. 5.]
 [4. 0. 9.]]

feat_vals_arr
[[1.         1.         0.9578554 ]
 [1.         1.         0.9417463 ]
 [1.         1.         0.65312517]
 [1.         1.         0.7425048 ]
 [1.         1.         0.6467193 ]
 [1.         1.         0.4838316 ]
 [1.         1.         0.01414096]
 [1.         1.         0.57208407]
 [1.         1.         0.06287198]]

emb vectors
tf.Tensor(
[[[-0.00250721  0.03223243 -0.02837677  0.02888547]
  [-0.03880321  0.03691218  0.02859639  0.0448879 ]
  [-0.03320575  0.02791285 -0.03738778 -0.03431834]]

 [[-0.00250721  0.03223243 -0.02837677  0.02888547]
  [-0.00303785  0.04685927  0.03928286 -0.03207158]
  [-0.0326473   0.02744341 -0.036759   -0.03374117]]

 [[-0.03880321  0.03691218  0.02859639  0.0448879 ]
  [-0.04194355  0.03691251 -0.02264855 -0.02871122]
  [ 0.00264363  0.03192767  0.00278697  0.02896227]]

 [[ 0.0323647   0.02443125  0.044909

In [6]:
class FMComponent(tf.keras.layers.Layer):
    def __init__(self,feat_dim,field_num,emb_dim,*args,**kwargs):
        super(FMComponent,self).__init__(*args,**kwargs)
        self.feat_dim=feat_dim
        self.field_num=field_num
        self.emb_dim=emb_dim

    def build(self, input_shape):
        self.w=tf.Variable(initial_value=tf.random.truncated_normal(shape=[self.feat_dim, self.emb_dim]))

    def call(self, inputs, **kwargs):
        """

        :param inputs: (raw_input_batch,emb_vectors) ，其中raw_input_batch是feat_indices_batch,feat_value_batch 用于计算一阶term
            后者是emb_layer的输出
        :param kwargs:
        :return:
        """
        raw_input_batch,emb_vectors=inputs # emb_vectors: [batch_size, field_num, emb_dim]
        # feat_indices_batch: [batch_size, field_num]
        # feat_value_batch: [batch_size, field_num]
        feat_indices_batch,feat_value_batch=raw_input_batch

        # first order term
        # 使用feat_indices找到embedding_lookup快速找到field_num个权重然后做相乘
        # 例如，如果一个样本x，他在特征维度1、3、5上有取值，那么他的feat_indices=[1,3,5]。那只需要从self.w找到第1、3、5个数就可以了
        # 这样的计算方法更加快速
        # 改进1 一阶输出也是一个向量而非一个标量，这就要求self.w的shape为[self.feat_dim, self.emb_dim]
        weights=tf.nn.embedding_lookup(params=self.w,ids=tf.cast(feat_indices_batch,tf.int32)) # [batch_size, field_num, self.emb_dim]
        # 需要对feat_value_batch扩充一下，不然无法进行broadcast
        first_order_term = tf.multiply(tf.expand_dims(feat_value_batch,axis=2),weights) # [batch_size, field_num, emb_dim]
        first_order_term = tf.reduce_sum(first_order_term,axis=1) # [batch_size, emb_dim]

        # second order term
        # 下面这个是fm算法的优化算法 和平方减去平方和
        sum_square=tf.square(tf.reduce_sum(emb_vectors,axis=1)) # [batch_size, emb_dim]
        square_sum=tf.reduce_sum(tf.square(emb_vectors),axis=1) # [batch_size, emb_dim]

        second_order_term=1/2*tf.subtract(sum_square,square_sum) # [batch_size, emb_dim]
        y_fm=first_order_term+second_order_term
        return y_fm

feat_indices_arr=[np.random.choice(range(10),size=[1,3],replace=False) for _ in range(9)]
feat_indices_arr=np.concatenate(feat_indices_arr,axis=0).astype(np.float32)
feat_vals_arr=np.concatenate((np.ones(shape=[9,2]),
                              np.random.random(size=[9,1])),axis=1).astype(np.float32)

input_ds=tf.data.Dataset.from_tensor_slices((feat_indices_arr,feat_vals_arr))
batched_ds=input_ds.batch(5)
iterator=iter(batched_ds)
input_batch=next(iterator)


print("\nemb vectors")
emb_layer=EmbeddingLayer(feat_dim=10,field_num=3,emb_dim=4)
emb_vectors=emb_layer(input_batch)
print(emb_vectors)

fm_component=FMComponent(feat_dim=10,field_num=3,emb_dim=4)
fm_inputs=(input_batch,emb_vectors)
fm_outputs=fm_component(fm_inputs)
print("\nfm_outputs")
print(fm_outputs)



emb vectors
tf.Tensor(
[[[ 0.01574266  0.03361524 -0.02441003  0.03478476]
  [ 0.03548969 -0.0261873   0.04189844 -0.00808696]
  [ 0.00147651  0.01941362  0.01325359 -0.01578611]]

 [[ 0.01574266  0.03361524 -0.02441003  0.03478476]
  [-0.0356837   0.01019305  0.0449813   0.02894637]
  [-0.01997713  0.00182355 -0.0290976   0.00926884]]

 [[-0.02286857  0.00208749 -0.03330912  0.01061039]
  [ 0.00792656 -0.04649333 -0.01925454 -0.03074418]
  [ 0.00253944  0.03338931  0.02279472 -0.02715039]]

 [[ 0.00792656 -0.04649333 -0.01925454 -0.03074418]
  [ 0.01095372 -0.01040577 -0.00403966 -0.0331562 ]
  [ 0.01182     0.00274881  0.00316286  0.01265087]]

 [[ 0.00293436  0.03858181  0.02633962 -0.03137266]
  [ 0.00792656 -0.04649333 -0.01925454 -0.03074418]
  [-0.00686075  0.00195977  0.00864836  0.00556539]]], shape=(5, 3, 4), dtype=float32)

fm_outputs
tf.Tensor(
[[ 0.17007205  0.30602217  1.2417642   0.85751283]
 [-2.3149729   1.5989451   2.3062763   0.10335954]
 [-1.5165918  -0.9349031  -0

In [8]:
class DeepComponent(tf.keras.layers.Layer):
    def __init__(self,deep_units_list,*args,**kwargs):
        super(DeepComponent,self).__init__(*args,**kwargs)
        self.deep_layers=list()
        for deep_units in deep_units_list:
            self.deep_layers.append(tf.keras.layers.Dense(units=deep_units,activation=tf.nn.relu))

    def call(self, inputs, **kwargs):
        for deep_layer in self.deep_layers:
            inputs=deep_layer(inputs)
        return inputs

In [10]:

class DeepFM(tf.keras.Model):
    def __init__(self,feat_dim,field_num,emb_dim,deep_units_list,scoring_units=2,*args,**kwargs):
        super(DeepFM,self).__init__(*args,**kwargs)

        self.emb_layer=EmbeddingLayer(feat_dim=feat_dim,field_num=field_num,emb_dim=emb_dim)
        self.fm_component=FMComponent(feat_dim=feat_dim,field_num=field_num,emb_dim=emb_dim)
        self.deep_component=DeepComponent(deep_units_list=deep_units_list)
        self.scoring_layer=tf.keras.layers.Dense(units=scoring_units,activation=None)

    def call(self, inputs, training=None, mask=None):
        emb_vectors=self.emb_layer(inputs)

        fm_inputs=(inputs,emb_vectors)
        y_fm=self.fm_component(fm_inputs)

        deep_inputs=tf.reshape(emb_vectors,shape=[emb_vectors.shape[0],-1])
        y_deep=self.deep_component(deep_inputs)
        y = self.scoring_layer(tf.concat((y_fm,y_deep),axis=1))

        return y


feat_indices_arr=[np.random.choice(range(10),size=[1,3],replace=False) for _ in range(9)]
feat_indices_arr=np.concatenate(feat_indices_arr,axis=0).astype(np.float32)
feat_vals_arr=np.concatenate((np.ones(shape=[9,2]),
                              np.random.random(size=[9,1])),axis=1).astype(np.float32)

input_ds=tf.data.Dataset.from_tensor_slices((feat_indices_arr,feat_vals_arr))
batched_ds=input_ds.batch(5)
iterator=iter(batched_ds)
input_batch=next(iterator)

deep_fm_model=DeepFM(feat_dim=10,field_num=3,emb_dim=4,deep_units_list=[10,8])
deep_fm_model(input_batch)



<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[0.24492802, 0.755072  ],
       [0.5246791 , 0.4753209 ],
       [0.39354166, 0.6064583 ],
       [0.5969774 , 0.40302256],
       [0.2637145 , 0.73628557]], dtype=float32)>

[[0.67044261 0.75686147]
 [0.14368512 0.98200156]
 [0.7414385  0.31037816]]
tf.Tensor(
[[0.47840872 0.52159128]
 [0.30188948 0.69811052]
 [0.60612684 0.39387316]], shape=(3, 2), dtype=float64)
